In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

from music21 import midi
from music21 import note, stream, duration
from music21 import converter

from models.GAN import MuseGAN

from utils.loaders import load_music

from keras.models import load_model

Using TensorFlow backend.


In [2]:
parent_folder = 'New_Music'    #Set up the data folders
run_num = '0001'
track_name1 = 'chorales'
file_name = 'Jsb16thSeparated.npz'
folder_in = 'run/{}/'.format(parent_folder)
folder_in += '_'.join([run_num, track_name1])


## Parameters Initialized

In [3]:
batch_in = 64
bars_num = 2
time_steps_per_bar = 16
n_notes = 84
num_of_tracks = 4

binary_data, data_code, raw = load_music(track_name1, file_name, bars_num, time_steps_per_bar)
# data_binary = np.squeeze(data_binary)

In [4]:
gan_model = MuseGAN(input_dim = binary_data.shape[1:]
        , critic_learning_rate = 0.001
        , generator_learning_rate = 0.001
        , optimiser = 'adam'
        , grad_weight = 10
        , z_dim = 32
        , batch_size = batch_in
        , n_tracks = num_of_tracks
        , n_bars = bars_num
        , n_steps_per_bar = time_steps_per_bar
        , n_pitches = n_notes
        )


W0505 21:20:20.393235 13108 deprecation_wrapper.py:119] From C:\Users\Arjun\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0505 21:20:20.409224 13108 deprecation_wrapper.py:119] From C:\Users\Arjun\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0505 21:20:20.410202 13108 deprecation_wrapper.py:119] From C:\Users\Arjun\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0505 21:20:20.596709 13108 deprecation_wrapper.py:119] From C:\Users\Arjun\anaconda3\envs\generative\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session inst

In [5]:
gan_model.load_weights(folder_in, None)

In [6]:
gan_model.generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
melody_input (InputLayer)       (None, 4, 32)        0                                            
__________________________________________________________________________________________________
chords_input (InputLayer)       (None, 32)           0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 32)           0           melody_input[0][0]               
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 32)           0           melody_input[0][0]               
__________________________________________________________________________________________________
lambda_3 (

In [7]:
gan_model.critic.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
critic_input (InputLayer)    (None, 2, 16, 84, 4)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 16, 84, 128)    1152      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1, 16, 84, 128)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 16, 84, 128)    16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1, 16, 84, 128)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 16, 7, 128)     196736    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1, 16, 7, 128)     0         
__________

# Sample score

In [8]:
chords_noise = np.random.normal(0, 1, (1, gan_model.z_dim))
style_noise = np.random.normal(0, 1, (1, gan_model.z_dim))
melody_noise = np.random.normal(0, 1, (1, gan_model.n_tracks, gan_model.z_dim))
groove_noise = np.random.normal(0, 1, (1, gan_model.n_tracks, gan_model.z_dim))

In [9]:
gen_scores = gan_model.generator.predict([chords_noise, style_noise, melody_noise, groove_noise])

In [10]:

np.argmax(gen_scores[0,0,0:4,:,3], axis = 1)

array([55, 65, 56, 76], dtype=int64)

In [11]:
gen_scores[0,0,0:4,60,3] = 0.02347812

In [12]:
filename = 'example'
gan_model.notes_to_midi(folder_in, gen_scores, filename)
gen_score = converter.parse(os.path.join(folder_in, 'data_storage/{}.midi'.format(filename)))

#Check the data_storage folder for the sample audio file.

# Closest match

In [13]:
def find_closest(binary_data, score):
    current_dist = 99999999
    current_i = -1
    for i, d in enumerate(binary_data):
        dist = np.sqrt(np.sum(pow((d - score),2)))
        if dist < current_dist:
            current_i = i
            current_dist = dist
        
    return current_i
    

In [14]:
closest_idx = find_closest(binary_data, gen_scores[0])
closest_data = binary_data[[closest_idx]]
print(closest_idx)

113


In [15]:
filename = 'closest'
gan_model.notes_to_midi(folder_in, closest_data,filename)
closest_score = converter.parse(os.path.join(folder_in, 'data_storage/{}.midi'.format(filename)))

print('closest')

#Check the data_storage folder for the closest matched audio file.

closest


# change in chords noise

In [16]:
chords_noise_2 = 5 * np.ones((1, gan_model.z_dim))

In [17]:
chords_scores = gan_model.generator.predict([chords_noise_2, style_noise, melody_noise, groove_noise])

In [18]:
filename = 'changing_chords'
gan_model.notes_to_midi(folder_in, chords_scores, filename)
chords_score = converter.parse(os.path.join(folder_in, 'data_storage/{}.midi'.format(filename)))
print('original')

print('chords noise changed')
#Check the data_storage folder for the changing_chords audio file.

original
chords noise changed


# change in style noise

In [19]:
style_noise_2 = 5 * np.ones((1, gan_model.z_dim))

In [20]:
style_scores = gan_model.generator.predict([chords_noise, style_noise_2, melody_noise, groove_noise])

In [21]:
filename = 'changing_style'
gan_model.notes_to_midi(folder_in, style_scores, filename)
style_score = converter.parse(os.path.join(folder_in, 'data_storage/{}.midi'.format(filename)))
print('original')

print('style noise changed')

#Check the data_storage folder for the changing_chords audio file.

original
style noise changed


# changing the melody noise

In [22]:
melody_noise_2 = np.copy(melody_noise)
melody_noise_2[0,0,:] = 5 * np.ones(gan_model.z_dim) 

In [23]:
melody_scores = gan_model.generator.predict([chords_noise, style_noise, melody_noise_2, groove_noise])

In [24]:
filename = 'changing_melody'
gan_model.notes_to_midi(folder_in, melody_scores, filename)
melody_score = converter.parse(os.path.join(folder_in, 'data_storage/{}.midi'.format(filename)))

print('melody noise changed')
#Check the data_storage folder for the changing_melody audio file.

melody noise changed


# change in groove noise

In [25]:
groove_noise_2 = np.copy(groove_noise)
groove_noise_2[0,3,:] = 5 * np.ones(gan_model.z_dim)

In [26]:
groove_scores = gan_model.generator.predict([chords_noise, style_noise, melody_noise, groove_noise_2])

In [27]:
filename = 'changing_groove'
gan_model.notes_to_midi(folder_in, groove_scores, filename)
groove_score = converter.parse(os.path.join(folder_in, 'data_storage/{}.midi'.format(filename)))

print('groove noise changed')
#Check the data_storage folder for the changing_groove audio file.

groove noise changed
